
Import Libraries

In [67]:
#IMPORTING LIBRARIES
import pandas as pd
import numpy as np
import os

In [69]:
#Creating path
path= r'C:\Users\beaac\Instacart Basket 25'

In [71]:
df_ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data' , 'Prepared Data' , 'orders_products_merged_derived.pkl'))

In [73]:
df_ords_prods_merge.shape

(32404859, 21)

Aggregated mean calculation for entire Dataframe

In [76]:
#grouping by 'department id' and 'order number'
df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


How the results for the entire dataframe differ from those of the subset?

The entire dataframe has higher mean order numbers across departments thant the subset. Department 21 has the higest mean in both. overall, the entire dataframe reveals ore frequent ordering patterns across departments compared to the subset.


Create a loyalty flag

In [81]:
#aggregating Data with Transform fuction
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\beaac\AppData\Local\Temp\ipykernel_20528\3890110530.py:2: FutureWarning: The provided callable <function max at 0x000002F97920C4A0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [83]:
#checking the output
df_ords_prods_merge.head(15)

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,aisle_id,department_id,prices,match,prices_range_loc,price_range_loc,busiest_day,busiest_days,busiest_period_of_day,max_order
0,0,2539329,1,1,2,8,0.0,196,1,0,...,77,7,9.0,both,Mid-range product,Low-range product,Regularly busy,Regularly busy,Average orders,10
1,0,2539329,1,1,2,8,0.0,14084,2,0,...,91,16,12.5,both,Mid-range product,Low-range product,Regularly busy,Regularly busy,Average orders,10
2,0,2539329,1,1,2,8,0.0,12427,3,0,...,23,19,4.4,both,Low-range product,Low-range product,Regularly busy,Regularly busy,Average orders,10
3,0,2539329,1,1,2,8,0.0,26088,4,0,...,23,19,4.7,both,Low-range product,Low-range product,Regularly busy,Regularly busy,Average orders,10
4,0,2539329,1,1,2,8,0.0,26405,5,0,...,54,17,1.0,both,Low-range product,Low-range product,Regularly busy,Regularly busy,Average orders,10
5,1,2398795,1,2,3,7,15.0,196,1,1,...,77,7,9.0,both,Mid-range product,Low-range product,Regularly busy,Slowest days,Average orders,10
6,1,2398795,1,2,3,7,15.0,10258,2,0,...,117,19,3.0,both,Low-range product,Low-range product,Regularly busy,Slowest days,Average orders,10
7,1,2398795,1,2,3,7,15.0,12427,3,1,...,23,19,4.4,both,Low-range product,Low-range product,Regularly busy,Slowest days,Average orders,10
8,1,2398795,1,2,3,7,15.0,13176,4,0,...,24,4,10.3,both,Mid-range product,Low-range product,Regularly busy,Slowest days,Average orders,10
9,1,2398795,1,2,3,7,15.0,26088,5,1,...,23,19,4.7,both,Low-range product,Low-range product,Regularly busy,Slowest days,Average orders,10


In [85]:
#creating a loyalty label flag
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'


In [86]:
#checking values
df_ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64


Analyze Spending Habits by loyalty Category

In [90]:
#Determining Basic Statistics
df_ords_prods_merge.groupby('loyalty_flag').agg({'prices':['mean','min','max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular customer  12.495717  1.0  99999.0


Average Spending per Order:

Loyal customers spend the least on average, potentially indicating that they make frequent, smaller purchases.

New customers spend the most per order, which could mean they are trying out more expensive products or buying in bulk.

Create a Spending Flag

In [94]:
#Aggregation Data with transfrm function
df_ords_prods_merge['avg_product_price']= df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\beaac\AppData\Local\Temp\ipykernel_20528\3232195486.py:2: FutureWarning: The provided callable <function mean at 0x000002F97920CEA0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_ords_prods_merge['avg_product_price']= df_ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)


In [96]:
#Creating a 'spenidng label flag'
df_ords_prods_merge.loc[df_ords_prods_merge['avg_product_price'] >= 10, 'spending_flag'] = 'High spender'
df_ords_prods_merge.loc[df_ords_prods_merge['avg_product_price'] < 10, 'spending_flag'] = 'Low spender'

In [97]:
#examining value count on speending_flag
df_ords_prods_merge['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

In [100]:
# Checking Releavent Columns
df_ords_prods_merge[['user_id', 'avg_product_price', 'spending_flag']].head(60)

,user_id,avg_product_price,spending_flag
0,1,6.367797,Low spender
1,1,6.367797,Low spender
2,1,6.367797,Low spender
3,1,6.367797,Low spender
4,1,6.367797,Low spender
5,1,6.367797,Low spender
6,1,6.367797,Low spender
7,1,6.367797,Low spender
8,1,6.367797,Low spender
9,1,6.367797,Low spender


Create an Order Frequency Flag

In [103]:
# Aggregating Data with transform() Function
df_ords_prods_merge['median_days_between_orders'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\beaac\AppData\Local\Temp\ipykernel_20528\4152215344.py:2: FutureWarning: The provided callable <function median at 0x000002F979423B00> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_ords_prods_merge['median_days_between_orders'] = df_ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [105]:
# Checking Relevant Columns 
df_ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders']].head(65)

,user_id,days_since_prior_order,median_days_between_orders
0,1,0.0,20.0
1,1,0.0,20.0
2,1,0.0,20.0
3,1,0.0,20.0
4,1,0.0,20.0
...,...,...,...
60,2,0.0,13.0
61,2,0.0,13.0
62,2,0.0,13.0
63,2,0.0,13.0


In [107]:
# Creating a 'Order Frequency' Label Flag using loc Function
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_between_orders'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['median_days_between_orders'] <= 20) & (df_ords_prods_merge['median_days_between_orders'] > 10), 'order_frequency_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_between_orders'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [109]:
# Examining Value Counts on 'order_frequency_flag'
df_ords_prods_merge['order_frequency_flag'].value_counts(dropna = False)

order_frequency_flag
Frequent customer        22796659
Regular customer          6921472
Non-frequent customer     2686728
Name: count, dtype: int64

In [111]:
# Checking Relevant Columns of Updated Dataframe
df_ords_prods_merge[['user_id', 'days_since_prior_order', 'median_days_between_orders', 'order_frequency_flag']].head(65)

,user_id,days_since_prior_order,median_days_between_orders,order_frequency_flag
0,1,0.0,20.0,Regular customer
1,1,0.0,20.0,Regular customer
2,1,0.0,20.0,Regular customer
3,1,0.0,20.0,Regular customer
4,1,0.0,20.0,Regular customer
...,...,...,...,...
60,2,0.0,13.0,Regular customer
61,2,0.0,13.0,Regular customer
62,2,0.0,13.0,Regular customer
63,2,0.0,13.0,Regular customer


Export dataframe in pickle format

In [118]:
df_ords_prods_merge.loc[df_ords_prods_merge['prices'] >100, 'prices'] = np.nan

In [120]:
# Exporting the Dataframe
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_grouped.pkl'))